In [1]:
from dateutil import parser
from pytz import timezone


def filter_on_timezones(filename,cur):

    list_timezones = []
    list_timezones.append("Eastern Time (US & Canada)")
    list_timezones.append("Pacific Time (US & Canada)")
    list_timezones.append("Central Time (US & Canada)")
    list_timezones.append("Mountain Time (US & Canada)")
    #geolocator = Nominatim()

    count_all_tweets = 0
    fileh = gzip.open(filename,'rb')
    while True:
        
        user_timezone = ""
        
        try: 
            x = fileh.readline()
            x = x.rstrip()
            if not x: 
                break
            data = json.loads(x)
            #pprint(data)
            if 'id_str' not in data:
                continue
                
            id_str = data['id_str']
            count_all_tweets = count_all_tweets+1

            query = "SELECT EXISTS(SELECT 1 FROM all_buckets WHERE tweet_id = "+id_str+" LIMIT 1)"

            cur.execute(query)
            row = cur.fetchone()
            res = row[0]

            if res==1 :
                #print "already saved " + str(id_str)
                continue
                
            created_at = data['created_at']

            date = parser.parse(created_at)
            #print date
            
            user_timezone = data['user']['time_zone']
                        
            map_timezones = get_map_timezones()
            
            if user_timezone:
                args = []
                args.append((id_str))
                args.append(str(created_at))
                
                if user_timezone in map_timezones.keys():
                    user_timezone = map_timezones[user_timezone.strip()]
                    
                local_tzone = user_timezone.replace(" Time (US & Canada)","")
                local_date = date.astimezone(timezone("US/"+local_tzone))

                args.append(str(local_date))
                args.append(str(user_timezone))
                
                utc_ts = parser.parse(created_at)
                #print utc_ts
                
                utc_date_time = utc_ts.date().strftime("%Y-%m-%d")
                
                #print utc_date_time
                
                utc_day = utc_ts.day
                utc_hour = utc_ts.hour
                utc_mins = utc_ts.minute
                
                args.append(int(utc_day))
                args.append(int(utc_hour))
                args.append(int(utc_mins))
                
                args.append(int(get_bucket_number(utc_hour,utc_mins)))
                    
                day = local_date.day
                hour = local_date.hour
                minutes = local_date.minute
                
                args.append(int(day))
                args.append(int(hour))
                args.append(int(minutes))
                
                args.append(int(get_bucket_number(hour,minutes)))
                args.append(utc_date_time)
                
                if user_timezone in list_timezones: 
                    args = tuple(args)
                    #print args

                    try:
                        cur.execute('''INSERT INTO all_buckets 
                                    values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''',args)

                        conn.commit()
                    except Exception as e1:
                        #print e1
                        #traceback.print_exc()
                        conn.rollback()
                        #break


        except Exception as e:
            #print e
            #traceback.print_exc()
            continue
        #break
    #print count_all_tweets

In [2]:
def get_bucket_number(hour,minutes):
    
    if 0 <= minutes <= 4:
        bucket = (hour*12) + 1
    elif 5<= minutes <= 9:
        bucket = (hour*12) + 2
    elif 10<= minutes <= 14:
        bucket = (hour*12) + 3
    elif 15<= minutes <= 19:
        bucket = (hour*12) + 4
    elif 20<= minutes <= 24:
        bucket = (hour*12) + 5
    elif 25<= minutes <= 29:
        bucket = (hour*12) + 6
    elif 30<= minutes <= 34:
        bucket = (hour*12) + 7
    elif 35<= minutes <= 39:
        bucket = (hour*12) + 8
    elif 40<= minutes <= 44:
        bucket = (hour*12) + 9
    elif 45<= minutes <= 49:
        bucket = (hour*12) + 10
    elif 50<= minutes <= 54:
        bucket = (hour*12) + 11
    elif 55<= minutes <= 59:
        bucket = (hour*12) + 12

    return bucket


In [3]:
def get_map_timezones():
    
    map_timezoneid_timezonename = {}

    map_timezoneid_timezonename["America/New_York"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["America/Atikokan"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["America/Boise"] = "Mountain Time (US & Canada)"
    map_timezoneid_timezonename["America/Cambridge_Bay"] = "Mountain Time (US & Canada)"
    map_timezoneid_timezonename["America/Cancun"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["America/Chicago"] = "Central Time (US & Canada)"
    map_timezoneid_timezonename["America/Denver"] = "Mountain Time (US & Canada)"
    map_timezoneid_timezonename["America/Detroit"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["America/Kentucky"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["America/Los_Angeles"] = "Pacific Time (US & Canada)"
    map_timezoneid_timezonename["America/New_York"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["America/North_Dakota"]="Central Time (US & Canada)"
    map_timezoneid_timezonename["America/Phoenix"]= "Mountain Time (US & Canada)"
    map_timezoneid_timezonename["America/Regina"]= "Central Time (US & Canada)"
    map_timezoneid_timezonename["America/El_Salvador"] = "Central Time (US & Canada)"
    map_timezoneid_timezonename["CST"] = "Central Time (US & Canada)"
    map_timezoneid_timezonename["PST"] = "Pacific Time (US & Canada)"
    map_timezoneid_timezonename["EST"] = "Eastern Time (US & Canada)"
    map_timezoneid_timezonename["MST"] = "Mountain Time (US & Canada)"
    map_timezoneid_timezonename["America/Tijuana"] = "Pacific Time (US & Canada)"
    map_timezoneid_timezonename["Indiana (East)"] = "Eastern Time (US & Canada)"
    
    return map_timezoneid_timezonename

In [4]:
def process_tweets(path,cursor):
    
    try:
        for filename in os.listdir(path):

            if filename.startswith('.'):
                continue

            filename = path+filename
            print filename
            filter_on_timezones(filename,cursor)
            #break
    except Exception as e:
        print e
        traceback.print_exc()

        

In [5]:
import os
import gzip
import traceback
import json
from pprint import pprint
import MySQLdb

conn = MySQLdb.connect(host= "localhost",
                      user="root",
                      passwd="sindhu",
                      db="sp")



cursor = conn.cursor()


if __name__ == '__main__':

    try:
    
        process_tweets("../dataset/allfiles/",cursor)

    except Exception as e:
        traceback.print_exc()
        print "Error: unable to start thread"


../dataset/allfiles/2015_06_03_19_onepercent.txt.gz
../dataset/allfiles/2015_06_03_20_onepercent.txt.gz
../dataset/allfiles/2015_06_03_21_onepercent.txt.gz
../dataset/allfiles/2015_06_03_22_onepercent.txt.gz
../dataset/allfiles/2015_06_03_23_onepercent.txt.gz
../dataset/allfiles/2015_06_04_00_onepercent.txt.gz
../dataset/allfiles/2015_06_04_01_onepercent.txt.gz
../dataset/allfiles/2015_06_04_02_onepercent.txt.gz
../dataset/allfiles/2015_06_04_03_onepercent.txt.gz
../dataset/allfiles/2015_06_04_04_onepercent.txt.gz
../dataset/allfiles/2015_06_04_05_onepercent.txt.gz
../dataset/allfiles/2015_06_04_06_onepercent.txt.gz
../dataset/allfiles/2015_06_04_07_onepercent.txt.gz
../dataset/allfiles/2015_06_04_08_onepercent.txt.gz
../dataset/allfiles/2015_06_04_09_onepercent.txt.gz
../dataset/allfiles/2015_06_04_10_onepercent.txt.gz
../dataset/allfiles/2015_06_04_11_onepercent.txt.gz
../dataset/allfiles/2015_06_04_12_onepercent.txt.gz
../dataset/allfiles/2015_06_04_13_onepercent.txt.gz
../dataset/a